In [1]:
import pandas as pd
from flask import Flask, request, jsonify
import threading
import requests
import logging
import os
import json

In [2]:
os.environ['PROBE_COMPANY_BASE_URL'] = "https://api.probe42.in/probe_pro_sandbox/companies"
os.environ['PROBE_LLP_BASE_URL'] = "https://api.probe42.in/probe_pro_sandbox/llps"
os.environ['PROBE_API_KEY'] = "1bIrZWuiQr51IfpvikwtK4WPM06ccagW871TqWLn"
os.environ['PROBE_BASE_URL'] = "https://api.probe42.in/probe_pro_sandbox"
os.environ['PROBE_SEARCH_URL'] = "https://api.probe42.in/probe_pro_sandbox/entities"
os.environ['PROBE_COMPANY_REPORT_URL'] = "https://api.probe42.in/probe_reports_sandbox/companies"
os.environ['SCOREME_GST_GET_REPORT'] = "https://sm-gst-sandbox.scoreme.in/gst/external/getgstreport"

In [3]:
bse_nse_df = pd.read_excel('BSE_NSE_directors.xlsx')
defaulters_df = pd.read_excel('Defaulters_directors.xlsx')
second_degree_df = pd.read_excel('2nd Degree Directors&Companies.xlsx')
politically_exposed_df = pd.read_excel('POLITICALLY EXPOSED PERSONS.xlsx')

In [4]:
bse_nse_directors = set()
defaulters_directors = set()
second_degree_defaulters = set()
second_degree_relatives = set()
politically_exposed_persons = set()
politically_exposed_relatives = set()

In [5]:
for col in bse_nse_df.columns[1:]:
    bse_nse_directors.update(bse_nse_df[col].dropna().str.strip().str.lower())

In [6]:
for col in defaulters_df.columns[1:]:
    defaulters_directors.update(defaulters_df[col].dropna().str.strip().str.lower())

In [7]:
second_degree_defaulters.update(second_degree_df.iloc[:, 0].dropna().str.strip().str.lower())
second_degree_relatives.update(second_degree_df.iloc[:, 1:].stack().dropna().str.strip().str.lower())

In [8]:
politically_exposed_persons.update(politically_exposed_df.iloc[:, 0].dropna().str.strip().str.lower())
politically_exposed_relatives.update(politically_exposed_df.iloc[:, 1:].stack().dropna().str.strip().str.lower())

In [9]:
def score_director(name):
    name = name.strip().lower()
    score = 0
    
    if name in bse_nse_directors:
        score += 10
    if name in defaulters_directors:
        score -= 10
    if name in second_degree_relatives:
        score -= 3
    if name in second_degree_defaulters:
        score -= 10
    if name in politically_exposed_persons:
        score -= 10
    if name in politically_exposed_relatives:
        score -= 10
    
    return score

In [10]:
app = Flask(__name__)

In [11]:
logging.basicConfig(level=logging.DEBUG)

def get_probe_by_name(search_company):
    try:
        if len(search_company) < 3:
            return {"error": "Search string must be at least 3 characters long"}, 400

        api_key = os.getenv("PROBE_API_KEY")
        headers = {"x-api-version": "1.3", "x-api-key": api_key}
        filters = {"nameStartsWith": search_company}
        params = {"limit": 25, "filters": json.dumps(filters)}
        url = os.getenv("PROBE_SEARCH_URL")
        
        response = requests.get(url, headers=headers, params=params)

        response_data = response.json()
        if response.status_code == 200:
            return response_data, 200
        else:
            return {"error": f"Failed to fetch company list, {response_data.get('message', '')}"}, response.status_code

    except Exception as e:
        logging.error(f"An error occurred while fetching company search result list: {e}")
        return {"error": str(e)}, 500

@app.route('/score_company', methods=['POST'])
def score_company():
    data = request.get_json()
    company_name = data.get('company_name')
    
    if not company_name:
        logging.error("Company name is required")
        return jsonify({"error": "Company name is required"}), 400
    
    try:
        # Fetching company details from Probe API
        company_data, status_code = get_probe_by_name(company_name)
        
        if status_code != 200:
            return jsonify(company_data), status_code
        
        # Getting the directors 
        directors = []
        for entity in company_data.get('items', []):
            directors.extend(entity.get('directors', []))
        
        scores = {}
        for director in directors:
            scores[director] = score_director(director)
        
        return jsonify(scores)
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data from Probe API: {e}")
        return jsonify({"error": "Failed to fetch data from Probe API"}), 500
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return jsonify({"error": "An unexpected error occurred"}), 500

In [12]:
def run_app():
    app.run(port=5000)

In [13]:
threading.Thread(target=run_app).start()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.probe42.in:443
DEBUG:urllib3.connectionpool:https://api.probe42.in:443 "GET /probe_pro_sandbox/entities?limit=25&filters=%7B%22nameStartsWith%22%3A+%22SAKHI+INDUSTRIES+LIMITED%22%7D HTTP/1.1" 200 2651
INFO:werkzeug:127.0.0.1 - - [03/Jun/2024 14:49:42] "POST /score_company HTTP/1.1" 200 -
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.probe42.in:443
DEBUG:urllib3.connectionpool:https://api.probe42.in:443 "GET /probe_pro_sandbox/entities?limit=25&filters=%7B%22nameStartsWith%22%3A+%22SAKHI+INDUSTRIES+LIMITED%22%7D HTTP/1.1" 200 2651
INFO:werkzeug:127.0.0.1 - - [03/Jun/2024 14:56:32] "POST /score_company HTTP/1.1" 200 -
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.probe42.in:443